In [1]:
import gurobipy as gp
from gurobipy import GRB

#import modules

In [2]:
import json
import networkx
from networkx.readwrite import json_graph
#opening json file for data
def read_graph_from_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
    return json_graph.adjacency_graph(data)

In [3]:
#import Washington data from json file (WA)
filepath = "C:/Users/crazy/OneDrive/Desktop/School/Operation Research/Final Project/"
filename = 'WA_county.json'

G = read_graph_from_json(filepath + filename)

In [4]:
for node in G.nodes: #for every node its gonna print and give us an idea of what the data looks like
    county_name = G.nodes[node]["NAMELSAD10"]
    county_population = G.nodes[node]['TOTPOP']
    print("Node",node,"represents",county_name,"County, which had a population of", county_population, "in the 2020 census")

Node 0 represents Yakima County County, which had a population of 256728 in the 2020 census
Node 1 represents Chelan County County, which had a population of 79074 in the 2020 census
Node 2 represents Kitsap County County, which had a population of 275611 in the 2020 census
Node 3 represents Lincoln County County, which had a population of 10876 in the 2020 census
Node 4 represents Mason County County, which had a population of 65726 in the 2020 census
Node 5 represents Pend Oreille County County, which had a population of 13401 in the 2020 census
Node 6 represents Skagit County County, which had a population of 129523 in the 2020 census
Node 7 represents Clallam County County, which had a population of 77155 in the 2020 census
Node 8 represents Ferry County County, which had a population of 7551 in the 2020 census
Node 9 represents Asotin County County, which had a population of 22285 in the 2020 census
Node 10 represents Franklin County County, which had a population of 96749 in the 

In [5]:
k = 10 #number of districts

m = gp.Model() #create model for district divisions by counties

x = m.addVars(G.nodes, k, vtype=GRB.BINARY)
l = m.addVar()    #smalles district population
u = m.addVar()    #largest district population


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-11


In [6]:
m.setObjective(u-l, GRB.MINIMIZE) #minimizing the smallest difference between smallest and largest

#adding constraints
#adds constraint for 1 district per county
#each country should be kept whole for requirements
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum( x[i,j] for j in range(k) ) == 1 for i in G.nodes ) 

# add constraints saying that each district has population at least y
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) >= l for j in range(k) )

# add constraints saying that each district has population at most z
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) <= u for j in range(k) )

m.update()

In [7]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 7840HS with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 59 rows, 392 columns and 1190 nonzeros
Model fingerprint: 0xddb3f514
Variable types: 2 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2828838.0000
Presolve time: 0.00s
Presolved: 59 rows, 392 columns, 1190 nonzeros
Variable types: 0 continuous, 392 integer (390 binary)

Root relaxation: objective 0.000000e+00, 67 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   15 2828838.00    0

In [8]:
print(G.nodes[3]["TOTPOP"]) #Trying to make sure the data works 
#returns total population for each county

10876


In [9]:
# print the absolute population deviation
print("The minimum required deviation is",m.objVal,"persons.") 

# retrieve the districts and their populations
districts = [ [ i for i in G.nodes if x[i,j].x > 0.5 ] for j in range(k) ]
district_county_names = [ [ G.nodes[i]['NAMELSAD10'] for i in district ] for district in districts ]
district_populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has these nodes =",districts[j],"and this population =",district_populations[j] )
    print("The corresponding county names are =",district_county_names[j] )
    print("")


The minimum required deviation is 1745333.0 persons.
District 0 has these nodes = [0, 14, 17, 27, 30, 38] and this population = 524360
The corresponding county names are = ['Yakima County', 'Cowlitz County', 'Lewis County', 'Adams County', 'Skamania County', 'Okanogan County']

District 1 has these nodes = [24] and this population = 2269675
The corresponding county names are = ['King County']

District 2 has these nodes = [11, 28, 36] and this population = 525051
The corresponding county names are = ['Clark County', 'San Juan County', 'Columbia County']

District 3 has these nodes = [29] and this population = 827957
The corresponding county names are = ['Snohomish County']

District 4 has these nodes = [2, 6, 8, 9, 18, 23] and this population = 524354
The corresponding county names are = ['Kitsap County', 'Skagit County', 'Ferry County', 'Asotin County', 'Douglas County', 'Stevens County']

District 5 has these nodes = [1, 5, 21, 25, 32, 33] and this population = 524425
The correspondi

In [10]:
#since the counties are not very evenly split it makes it infeasible to divide them into even deviations 
#based on counties alone
#the ideal deviation should be 
total=0
for i in G.nodes():
    total=total+G.nodes[i]["TOTPOP"]
    print(i,total)
print("ideal deviation",total/k)
#this shows the population should be about 770502 per district. 
#This means we will have to split districts in order to get a more even split


0 256728
1 335802
2 611413
3 622289
4 688015
5 701416
6 830939
7 908094
8 915645
9 937930
10 1034679
11 1537990
12 1582327
13 1605062
14 1715792
15 1738527
16 2033320
17 2115469
18 2158407
19 2385254
20 2418231
21 2625104
22 2673077
23 2719523
24 4989198
25 5051782
26 5054068
27 5074681
28 5092469
29 5920426
30 5932462
31 6031585
32 6118442
33 6194078
34 7115208
35 7119630
36 7123582
37 7662921
38 7705025
ideal deviation 770502.5


In [76]:
#Now I will try to use an altered model using tracts instead of counties
G2 = read_graph_from_json("C:/Users/crazy/OneDrive/Desktop/School/Operation Research/Final Project/WA_tracts.json")

m2 = gp.Model()

x2 = m2.addVars(G2.nodes, k, vtype=GRB.CONTINUOUS)
l2 = m2.addVar()    #smalles district population
u2 = m2.addVar()

In [17]:
m2.setObjective(u2-l2, GRB.MINIMIZE)

m2.addConstrs( gp.quicksum( x2[i,j] for j in range(k) ) == 1 for i in G2.nodes ) 

m2.addConstrs( gp.quicksum( G2.nodes[i]['TOTPOP'] * x2[i,j] for i in G2.nodes ) >= l2 for j in range(k) )

m2.addConstrs( gp.quicksum( G2.nodes[i]['TOTPOP'] * x2[i,j] for i in G2.nodes ) <= u2 for j in range(k) )

m2.update()

In [18]:
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 7840HS with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1478 rows, 14582 columns and 43500 nonzeros
Model fingerprint: 0x6f71036e
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 13 rows and 130 columns
Presolve time: 0.03s
Presolved: 1465 rows, 14452 columns, 43370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

    1292 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

   14279 PPushes remaining with PInf 0.0

In [22]:
# print the absolute population deviation
print("The minimum required deviation is",m2.objVal,"persons.")

# retrieve the districts and their populations
districts2 = [ [ i for i in G2.nodes if x2[i,j].x > 0.5 ] for j in range(k) ]
district_county_names2 = [ [ G2.nodes[i]['NAMELSAD10'] for i in district2 ] for district2 in districts2 ]
district_populations2 = [ sum( G2.nodes[i]['TOTPOP'] for i in district2 ) for district2 in districts2 ]
census_tracts = []
# print district info
for j in range(k):
    print("District",j,"has these nodes =",districts2[j],"and this population =",district_populations2[j] )
    print("The corresponding county names are =",district_county_names2[j] )
    census_tracts.append(district_populations2[j])
    print("")

The minimum required deviation is -2.3283064365386963e-10 persons.
District 0 has these nodes = [1, 6, 22, 36, 48, 58, 72, 84, 92, 105, 118, 132, 144, 161, 173, 184, 193, 197, 202, 214, 224, 237, 246, 262, 275, 287, 300, 311, 313, 325, 339, 341, 365, 373, 389, 397, 407, 419, 433, 449, 459, 464, 475, 488, 495, 501, 512, 520, 528, 545, 563, 572, 576, 586, 593, 611, 614, 617, 635, 644, 647, 663, 677, 696, 708, 730, 744, 759, 764, 769, 778, 785, 794, 804, 814, 828, 837, 850, 858, 874, 887, 894, 905, 916, 925, 939, 953, 965, 981, 988, 1002, 1010, 1025, 1032, 1044, 1053, 1061, 1077, 1088, 1100, 1112, 1122, 1128, 1132, 1144, 1154, 1155, 1160, 1171, 1183, 1184, 1192, 1200, 1202, 1206, 1218, 1229, 1238, 1245, 1255, 1264, 1273, 1280, 1292, 1306, 1319, 1328, 1344, 1360, 1369, 1382, 1395, 1407, 1419, 1426, 1435] and this population = 670646
The corresponding county names are = ['Census Tract 716.01', 'Census Tract 728', 'Census Tract 732', 'Census Tract 715.04', 'Census Tract 725.04', 'Census Trac

In [23]:
census_tracts
#this shows the deviation between them is very small with the biggest difference being between 
#674200-669759= 4441 people  this is less than the required 1% by the st.dev which would be around 6700
#It is worth noting that with this census tract data the population
#numbers do not match the current 2020 census data despite being listed as 2020
#this was the easiest method to find tract data however so it will have to work
#this does give me a much better deviation than the county split however
#But the model doesnt have connections enforced

[670646,
 670706,
 673714,
 674200,
 670214,
 670761,
 669774,
 674190,
 669759,
 670784]

In [28]:
#we can check which ones are connected using networkx

import networkx as nx
for district in districts2:
    print("Is district =", district, "connected?",nx.is_connected( G.subgraph( district ) ) )
#this shows none of my districts are connected
#however my objective for optimization of deviation while maintaining 
#census tracts/districts is completed. 
    

Is district = [1, 6, 22, 36, 48, 58, 72, 84, 92, 105, 118, 132, 144, 161, 173, 184, 193, 197, 202, 214, 224, 237, 246, 262, 275, 287, 300, 311, 313, 325, 339, 341, 365, 373, 389, 397, 407, 419, 433, 449, 459, 464, 475, 488, 495, 501, 512, 520, 528, 545, 563, 572, 576, 586, 593, 611, 614, 617, 635, 644, 647, 663, 677, 696, 708, 730, 744, 759, 764, 769, 778, 785, 794, 804, 814, 828, 837, 850, 858, 874, 887, 894, 905, 916, 925, 939, 953, 965, 981, 988, 1002, 1010, 1025, 1032, 1044, 1053, 1061, 1077, 1088, 1100, 1112, 1122, 1128, 1132, 1144, 1154, 1155, 1160, 1171, 1183, 1184, 1192, 1200, 1202, 1206, 1218, 1229, 1238, 1245, 1255, 1264, 1273, 1280, 1292, 1306, 1319, 1328, 1344, 1360, 1369, 1382, 1395, 1407, 1419, 1426, 1435] connected? False
Is district = [13, 20, 31, 43, 54, 68, 79, 91, 102, 125, 139, 148, 159, 170, 181, 189, 203, 215, 228, 243, 254, 264, 274, 284, 290, 308, 323, 331, 342, 360, 369, 383, 400, 409, 422, 436, 444, 453, 472, 482, 496, 509, 521, 532, 553, 555, 560, 566, 575, 5

In [42]:
import geopandas as gpd

In [72]:
#worth noting you need all tracts files for this command to work
df=gpd.read_file("C:/Users/crazy/OneDrive/Desktop/School/Operation Research/WA_tracts.shp")

print(G2)

Graph with 1458 nodes and 4061 edges


In [77]:
"""assignment = [-1 for u in G2.nodes]

for j in range(len(districts2)):
    
    for i in districts2[j]:
        
        geoID = G2.nodes[i]["GEOID10"]
                                   
        for u in G2.nodes:
            if geoID == df['GEOID20'][u]:
                assignment[u]=j
df['assignment'] = assignment
my_fig = df.plot(column='assignment').get_figure()
"""

'assignment = [-1 for u in G2.nodes]\n\nfor j in range(len(districts2)):\n    \n    for i in districts2[j]:\n        \n        geoID = G2.nodes[i]["GEOID10"]\n                                   \n        for u in G2.nodes:\n            if geoID == df[\'GEOID20\'][u]:\n                assignment[u]=j\ndf[\'assignment\'] = assignment\nmy_fig = df.plot(column=\'assignment\').get_figure()\n'